In [1]:
%run stdPackages.ipynb

# Read database

In [2]:
from pyDatabases.simpleDB_wheels import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['LoadVariation_E','LoadVariation_H','HourlyVariation'],
        'scalars': ['Scalars']}
db = read.dbFromWB(os.path.join(d['data'],'mGFPH.xlsx'), kwargs)
readSets(db)

100 hours:

In [3]:
[db.__setitem__(k,adj.rc_pd(db[k], db['h'][0:1000])) for k in db.variableDomains('h', types=['set','variable'])];

Time:

In [54]:
import time
times = {}
t0 = time.time()
# Initialize model:
m = mGFPH.mSimple(db) # initialize using the database 'db' from above.
times['setup'] = time.time()-t0
# Initialize blocks:
m.preSolve()
m.initBlocks()
times['initBlocks'] = time.time()-t0-sum(times.values())
# Compile model:
m.blocks.compileParameters()
times['compileParameters'] = time.time()-t0-sum(times.values())
m.blocks.settingsFromCompiled()
times['settingsFromCompiled'] = time.time()-t0-sum(times.values())
m.blocks.inferGlobalDomains()
times['inferGlobalDomains'] = time.time()-t0-sum(times.values())
m.blocks.getDenseArgs()
times['getDenseArgs'] = time.time()-t0-sum(times.values())
args = m.blocks.lp_args
times['lp_args'] = time.time()-t0-sum(times.values())
sol = optimize.linprog(method = m.method, **args)
times['sol'] = time.time()-t0-sum(times.values())
m.postSolve(sol)
times['postSolve'] = time.time()-t0-sum(times.values())
times

{'setup': 0.15721917152404785,
 'initBlocks': 1.7197597026824951,
 'compileParameters': 1.0757272243499756,
 'settingsFromCompiled': 0.0,
 'inferGlobalDomains': 0.25620031356811523,
 'getDenseArgs': 0.6539745330810547,
 'lp_args': 0.40880298614501953,
 'sol': 2.007124185562134,
 'postSolve': 0.5398390293121338}

Try and force some sparsity to the model and see what happens:

In [67]:
args['bounds']

array([[0.  ,  nan],
       [0.  ,  nan],
       [0.  ,  nan],
       ...,
       [0.  , 2.16],
       [0.  , 2.16],
       [0.  , 2.16]])

In [64]:
args['bounds'].shape

(42028, 2)

In [59]:
args['c'] = pd.arrays.SparseArray(args['c'], fill_value = 0)
args['l'] = pd.arras

In [60]:
sol = optimize.linprog(method = m.method, **args)